## 프롬프트

- 사용자와 언어 모델 간의 대화에서 질문이나 요청의 형태로 제시되는 입력문
- 모델이 어떤 유형의 응답을 제공할지 결정하는 데 중요한 역할을 한다.

1. 명확성과 구체성
    - 질문은 명확하고 구체적으로
2. 배경 정보를 포함
    - 모델이 문맥을 이해할 수 있도록 필요한 배경 정보 제공
3. 간결함
    - 불필요한 정보는 배제
4. 열린 질문 사용
    - 풍부한 답변을 기대할 수 있음
5. 명확한 목표 설정
6. 대화의 맥락에 적합한 언어와 문체 사용
    - 공식적인 보고서를 요청할 건데 평어를 쓰면 안된다

In [1]:
from langchain_core.prompts import PromptTemplate

# 'name'과 'age'라는 두 개의 변수를 사용하는 프롬프트 템플릿을 정의
template_text = "안녕하세요, 제 이름은 {name}이고, 나이는 {age}살입니다."

# PromptTemplate 인스턴스를 생성
prompt_template = PromptTemplate.from_template(template_text)

# 템플릿에 값을 채워서 프롬프트를 완성
filled_prompt = prompt_template.format(name="홍길동", age=30)

filled_prompt

'안녕하세요, 제 이름은 홍길동이고, 나이는 30살입니다.'

In [2]:
# 문자열 템플릿 결합 (PromptTemplate + PromptTemplate + 문자열)
# 그냥 프롬프트 문자열을 결합시킬 수 있다고 생각하면 됨
combined_prompt = (
              prompt_template
              + PromptTemplate.from_template("\n\n아버지를 아버지라 부를 수 없습니다.")
              + "\n\n{language}로 번역해주세요."
)

combined_prompt

PromptTemplate(input_variables=['age', 'language', 'name'], input_types={}, partial_variables={}, template='안녕하세요, 제 이름은 {name}이고, 나이는 {age}살입니다.\n\n아버지를 아버지라 부를 수 없습니다.\n\n{language}로 번역해주세요.')

In [3]:
combined_prompt.format(name="홍길동", age=30, language="영어")

'안녕하세요, 제 이름은 홍길동이고, 나이는 30살입니다.\n\n아버지를 아버지라 부를 수 없습니다.\n\n영어로 번역해주세요.'

In [4]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(model="gpt-4o-mini")
chain = combined_prompt | llm | StrOutputParser()
chain.invoke({"age":30, "language":"영어", "name":"홍길동"})

'Hello, my name is Hong Gil-dong, and I am 30 years old.\n\nI cannot call my father "father."'

- SystemMessage: 시스템의 기능을 설명합니다.
- HumanMessage: 사용자의 질문을 나타냅니다.
- AIMessage: AI 모델의 응답을 제공합니다.
- FunctionMessage: 특정 함수 호출의 결과를 나타냅니다.
- ToolMessage: 도구 호출의 결과를 나타냅니다.

In [5]:
# 2-튜플 형태의 메시지 목록으로 프롬프트 생성 (type, content)

from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "이 시스템은 천문학 질문에 답변할 수 있습니다."),
    ("user", "{user_input}"),
])

messages = chat_prompt.format_messages(user_input="태양계에서 가장 큰 행성은 무엇인가요?")
messages

[SystemMessage(content='이 시스템은 천문학 질문에 답변할 수 있습니다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='태양계에서 가장 큰 행성은 무엇인가요?', additional_kwargs={}, response_metadata={})]

In [6]:
from langchain_core.output_parsers import StrOutputParser

chain = chat_prompt | llm | StrOutputParser()

chain.invoke({"user_input": "태양계에서 가장 큰 행성은 무엇인가요?"})

'태양계에서 가장 큰 행성은 목성(Jupiter)입니다. 목성은 지름이 약 139,822킬로미터로, 태양계의 다른 모든 행성을 합친 것보다도 더 큽니다. 또한, 목성은 가스 거인으로 분류되며, 두꺼운 대기와 강력한 자기장을 가지고 있습니다.'

In [ ]:
# MessagePromptTemplate 활용

from langchain_core.prompts import SystemMessagePromptTemplate,  HumanMessagePromptTemplate

chat_prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template("이 시스템은 천문학 질문에 답변할 수 있습니다."),
        HumanMessagePromptTemplate.from_template("{user_input}"),
    ]
)

messages = chat_prompt.format_messages(user_input="태양계에서 가장 큰 행성은 무엇인가요?")
messages

In [ ]:
chain = chat_prompt | llm | StrOutputParser()

chain.invoke({"user_input": "태양계에서 가장 큰 행성은 무엇인가요?"})


## Few-shot Prompt

- 몇 가지 예시를 제공하여 특정 작업을 수행하도록 유도하는 기법

In [7]:
from langchain_core.prompts import PromptTemplate

example_prompt = PromptTemplate.from_template("질문: {question}\n{answer}")

In [8]:
# 예제 세트 제공

examples = [
    {
        "question": "지구의 대기 중 가장 많은 비율을 차지하는 기체는 무엇인가요?",
        "answer": "지구 대기의 약 78%를 차지하는 질소입니다."
    },
    {
        "question": "광합성에 필요한 주요 요소들은 무엇인가요?",
        "answer": "광합성에 필요한 주요 요소는 빛, 이산화탄소, 물입니다."
    },
    {
        "question": "피타고라스 정리를 설명해주세요.",
        "answer": "피타고라스 정리는 직각삼각형에서 빗변의 제곱이 다른 두 변의 제곱의 합과 같다는 것입니다."
    },
    {
        "question": "지구의 자전 주기는 얼마인가요?",
        "answer": "지구의 자전 주기는 약 24시간(정확히는 23시간 56분 4초)입니다."
    },
    {
        "question": "DNA의 기본 구조를 간단히 설명해주세요.",
        "answer": "DNA는 두 개의 폴리뉴클레오티드 사슬이 이중 나선 구조를 이루고 있습니다."
    },
    {
        "question": "원주율(π)의 정의는 무엇인가요?",
        "answer": "원주율(π)은 원의 지름에 대한 원의 둘레의 비율입니다."
    }
]

In [9]:
from langchain_core.prompts import FewShotPromptTemplate

# FewShotPromptTemplate을 생성합니다.
prompt = FewShotPromptTemplate(
    examples=examples,              # 사용할 예제들
    example_prompt=example_prompt,  # 예제 포맷팅에 사용할 템플릿
    suffix="질문: {input}",          # 예제 뒤에 추가될 접미사
    input_variables=["input"],      # 입력 변수 지정
)

# 새로운 질문에 대한 프롬프트를 생성하고 출력합니다.
print(prompt.invoke({"input": "화성의 표면이 붉은 이유는 무엇인가요?"}).to_string())

# 위 예시를 통해 지금은 그냥 "화성의 표면이~~" 를 프롬프트로 주었지만 앞에 "질문: " 이게 붙을 것임.

질문: 지구의 대기 중 가장 많은 비율을 차지하는 기체는 무엇인가요?
지구 대기의 약 78%를 차지하는 질소입니다.

질문: 광합성에 필요한 주요 요소들은 무엇인가요?
광합성에 필요한 주요 요소는 빛, 이산화탄소, 물입니다.

질문: 피타고라스 정리를 설명해주세요.
피타고라스 정리는 직각삼각형에서 빗변의 제곱이 다른 두 변의 제곱의 합과 같다는 것입니다.

질문: 지구의 자전 주기는 얼마인가요?
지구의 자전 주기는 약 24시간(정확히는 23시간 56분 4초)입니다.

질문: DNA의 기본 구조를 간단히 설명해주세요.
DNA는 두 개의 폴리뉴클레오티드 사슬이 이중 나선 구조를 이루고 있습니다.

질문: 원주율(π)의 정의는 무엇인가요?
원주율(π)은 원의 지름에 대한 원의 둘레의 비율입니다.

질문: 화성의 표면이 붉은 이유는 무엇인가요?


In [10]:
from langchain_chroma import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_openai import OpenAIEmbeddings

# 예시들 중 가장 유사한 녀석 하나(k = 1)를 골라주는 녀석
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,            # 사용할 예제들
    OpenAIEmbeddings(),  # 임베딩 모델
    Chroma,              # 벡터 저장소
    k=1,                 # 선택할 예제 수
)

# 새로운 질문에 대해 가장 유사한 예제를 선택합니다.
question = "화성의 표면이 붉은 이유는 무엇인가요?"
selected_examples = example_selector.select_examples({"question": question})
print(f"입력과 가장 유사한 예제: {question}")
for example in selected_examples:
    print("\n")
    for k, v in example.items():
        print(f"{k}: {v}")

입력과 가장 유사한 예제: 화성의 표면이 붉은 이유는 무엇인가요?


answer: 지구 대기의 약 78%를 차지하는 질소입니다.
question: 지구의 대기 중 가장 많은 비율을 차지하는 기체는 무엇인가요?


In [11]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

# 예제 정의
examples = [
    {"input": "지구의 대기 중 가장 많은 비율을 차지하는 기체는 무엇인가요?", "output": "질소입니다."},
    {"input": "광합성에 필요한 주요 요소들은 무엇인가요?", "output": "빛, 이산화탄소, 물입니다."},
]

# 예제 프롬프트 템플릿 정의
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)

# Few-shot 프롬프트 템플릿 생성
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

# 최종 프롬프트 템플릿 생성
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 과학과 수학에 대해 잘 아는 교육자입니다."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

# 모델과 체인 생성
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)
chain = final_prompt | model

# 모델에 질문하기
result = chain.invoke({"input": "지구의 자전 주기는 얼마인가요?"})
print(result.content)

지구의 자전 주기는 약 24시간, 즉 1일입니다. 정확히는 약 23시간 56분 4초로, 이를 '항성일'이라고 합니다. 그러나 우리가 일반적으로 사용하는 1일은 태양일로, 태양이 같은 위치에 돌아오는 시간을 기준으로 합니다.


In [14]:
from langchain_chroma import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_openai import OpenAIEmbeddings

# 더 많은 예제 추가
examples = [
    {"input": "지구의 대기 중 가장 많은 비율을 차지하는 기체는 무엇인가요?", "output": "질소입니다."},
    {"input": "광합성에 필요한 주요 요소들은 무엇인가요?", "output": "빛, 이산화탄소, 물입니다."},
    {"input": "피타고라스 정리를 설명해주세요.", "output": "직각삼각형에서 빗변의 제곱은 다른 두 변의 제곱의 합과 같습니다."},
    {"input": "DNA의 기본 구조를 간단히 설명해주세요.", "output": "DNA는 이중 나선 구조를 가진 핵산입니다."},
    {"input": "원주율(π)의 정의는 무엇인가요?", "output": "원의 둘레와 지름의 비율입니다."},
]

# 벡터 저장소 생성
to_vectorize = [" ".join(example.values()) for example in examples]
print(to_vectorize)

embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_texts(to_vectorize, embeddings, metadatas=examples)

# 예제 선택기 생성; 상위 2개만 선택한다
example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=2,
)

# Few-shot 프롬프트 템플릿 생성
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_selector=example_selector,
    example_prompt=ChatPromptTemplate.from_messages(
        [("human", "{input}"), ("ai", "{output}")]
    ),
)

# 최종 프롬프트 템플릿 생성
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 과학과 수학에 대해 잘 아는 교육자입니다."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)
 
# 모델과 체인 생성
chain = final_prompt | ChatOpenAI(model="gpt-4o-mini", temperature=0.0)

# 모델에 질문하기
result = chain.invoke({"input":"태양계에서 가장 큰 행성은 무엇인가요?"})
print(result.content)


['지구의 대기 중 가장 많은 비율을 차지하는 기체는 무엇인가요? 질소입니다.', '광합성에 필요한 주요 요소들은 무엇인가요? 빛, 이산화탄소, 물입니다.', '피타고라스 정리를 설명해주세요. 직각삼각형에서 빗변의 제곱은 다른 두 변의 제곱의 합과 같습니다.', 'DNA의 기본 구조를 간단히 설명해주세요. DNA는 이중 나선 구조를 가진 핵산입니다.', '원주율(π)의 정의는 무엇인가요? 원의 둘레와 지름의 비율입니다.']
태양계에서 가장 큰 행성은 목성(Jupiter)입니다. 목성은 지름이 약 142,984킬로미터로, 태양계의 다른 행성들보다 훨씬 큽니다.


## Partial Prompt

- 별건 아니고, 변수를 미리 설정하거나 동적으로 설정할 수 있다는 내용

In [15]:
from langchain_core.prompts import PromptTemplate

# 기본 프롬프트 템플릿 정의
prompt = PromptTemplate.from_template("지구의 {layer}에서 가장 흔한 원소는 {element}입니다.")

# 'layer' 변수에 '지각' 값을 미리 지정하여 부분 포맷팅
partial_prompt = prompt.partial(element="산소")

# 나머지 'element' 변수만 입력하여 완전한 문장 생성
print(partial_prompt.format(layer="지각"))

지구의 지각에서 가장 흔한 원소는 산소입니다.


In [16]:
# 프롬프트 초기화 시 부분 변수 지정
prompt = PromptTemplate(
    template="지구의 {layer}에서 가장 흔한 원소는 {element}입니다.",
    input_variables=["element"],  # 사용자 입력이 필요한 변수
    partial_variables={"layer": "맨틀"}  # 미리 지정된 부분 변수
)

# 남은 'element' 변수만 입력하여 문장 생성
print(prompt.format(element="규소"))

지구의 맨틀에서 가장 흔한 원소는 규소입니다.


In [17]:
from datetime import datetime

# 현재 시각을 가져와서 계절을 반환하는 함수 정의
def get_current_season():
    month = datetime.now().month
    if 3 <= month <= 5:
        return "봄"
    elif 6 <= month <= 8:
        return "여름"
    elif 9 <= month <= 11:
        return "가을"
    else:
        return "겨울"

# 함수를 사용한 부분 변수가 있는 프롬프트 템플릿 정의
prompt = PromptTemplate(
    template="{season}에 일어나는 대표적인 지구과학 현상은 {phenomenon}입니다.",
    input_variables=["phenomenon"],  # 사용자 입력이 필요한 변수
    partial_variables={"season": get_current_season}  # 함수를 통해 동적으로 값을 생성하는 부분 변수
)

# 'phenomenon' 변수만 입력하여 현재 계절에 맞는 문장 생성
print(prompt.format(phenomenon="꽃가루 증가"))


겨울에 일어나는 대표적인 지구과학 현상은 꽃가루 증가입니다.
